In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, date, timedelta
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
import torch

data= pd.read_csv('/Users/theobeevers/AppliedMachineLearning2024/final poject/all_data/train_full.csv')

data['datetime'] = pd.to_datetime(data['datetime'])
data['year'] = data['datetime'].dt.year
data['month'] = data['datetime'].dt.month
data['day'] = data['datetime'].dt.day
data['weekday'] = data['datetime'].dt.weekday
data['hour'] = data['datetime'].dt.hour
data.sort_values(by='datetime', inplace=True)

In [ ]:
one_company_data = data[data['prediction_unit_id'] == 1]

production_data = one_company_data[one_company_data['is_consumption'] == 0]

hold = production_data[['target', 'installed_capacity']].copy()

production_data['target'] = production_data['target']/production_data['installed_capacity']
production_data.drop(['installed_capacity'], axis=1, inplace=True)

consumption_data  = one_company_data[one_company_data['is_consumption'] == 1]

In [ ]:
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

categorical_features = ['county', 'product_type', 'hour', 'month', 'weekday', 'day', 'is_business']
numerical_features = production_data.columns.difference(categorical_features + ['target','datetime'])

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numerical_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

data_preprocessed = preprocessor.fit_transform(production_data)

In [ ]:
def create_sequences(X, y, sequence_length=24):
    X_seqs, y_seqs = [], []
    # Ensure the loop does not go beyond the point where a valid target exists
    for i in range(len(X) - sequence_length):
        X_seqs.append(X[i:i + sequence_length])
        y_seqs.append(y[i + sequence_length])
    return np.array(X_seqs), np.array(y_seqs)

In [ ]:
data_features = data_preprocessed 
data_targets = production_data['target'].values 

dates = production_data['datetime'].values 

split_idx = int(len(dates) * 0.5)

# Split the data into training and testing sets
X_train = data_features[:split_idx]
y_train = data_targets[:split_idx]
X_test = data_features[split_idx:]
y_test = data_targets[split_idx:]

# Create sequences for training and testing sets
X_train_seq, y_train_seq = create_sequences(X_train, y_train)
X_test_seq, y_test_seq = create_sequences(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

plt.figure(figsize=(15, 7))
plt.plot(dates[:split_idx], y_train, label='Training Data', color='blue')
plt.plot(dates[split_idx:], y_test, label='Testing Data', color='red', linestyle='--')

# Highlight the transition from training to testing
plt.axvline(x=dates[split_idx], color='gray', linestyle='--', label='Start of Test Data')

# Formatting the date on the x-axis for better readability
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Rotation

plt.title('Training and Testing Data Split')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.show()

In [ ]:
y_train_seq = y_train_seq.reshape(-1, 1)  
y_test_seq = y_test_seq.reshape(-1,1)

X_train_tensor = torch.tensor(X_train_seq, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train_seq, dtype=torch.float32)
X_val_tensor = torch.tensor(X_test_seq, dtype=torch.float32)
y_val_tensor = torch.tensor(y_test_seq, dtype=torch.float32)

In [ ]:
import torch.nn as nn

class GRUNet(nn.Module):
    def __init__(self, input_dim):
        super(GRUNet, self).__init__()
        self.gru = nn.GRU(input_dim, 50, 2, batch_first=True)  # 50 hidden units, 2 layers
        self.fc = nn.Linear(50, 1)  # Predicting one output per time step


    def forward(self, x):
        x, _ = self.gru(x)
        x = self.fc(x)
        x = x[:, -1, :]
        # Select only the last timestep for each sequence
        return x.unsqueeze(-1)

input_dim = X_train_tensor.shape[2]  # This is the number of features per time step


In [ ]:
import torch.optim as optim

model = GRUNet(input_dim)  # Ensure the input_dim is correctly set
criterion = nn.L1Loss()  # L1 Loss function for MAE
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

# Make sure your data tensors are also moved to the same device
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor = X_val_tensor.to(device)
y_val_tensor = y_val_tensor.to(device)

# Cast tensors to float16
# Ensure everything is in float16
# model = model.half()  # Convert model to float16
# X_train_tensor = X_train_tensor.half()
# y_train_tensor = y_train_tensor.half()
# X_val_tensor = X_val_tensor.half()
# y_val_tensor = y_val_tensor.half()

num_epochs = 50 # Define the number of epochs
# print("Check NaN or Inf in inputs:", torch.isnan(X_train_tensor).any(), torch.isinf(X_train_tensor).any())
# print("Check NaN or Inf in targets:", torch.isnan(y_train_tensor).any(), torch.isinf(y_train_tensor).any())

# print("Check NaN or Inf in inputs:", torch.isnan(X_val_tensor).any(), torch.isinf(X_val_tensor).any())
# print("Check NaN or Inf in targets:", torch.isnan(y_val_tensor).any(, torch.isinf(y_val_tensor).any())


# Check for NaNs in each feature
# nan_counts = pd.DataFrame(torch.isnan(X_train_tensor).sum(dim=0))
# print("NaN counts per feature:", nan_counts)
# nan_counts.to_csv('nan_counts')

train_losses = []
val_losses = []
for epoch in range(num_epochs):
    model.train()

    optimizer.zero_grad()
    outputs = model(X_train_tensor)
    outputs = outputs.squeeze(-1)

    loss = criterion(outputs, y_train_tensor)
    if not torch.isfinite(loss):
        print(f"Stopping training, loss has become {loss.item()}")
        break  # Early exit if loss is nan or inf
    loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1)
    optimizer.step()

    print(f'Epoch {epoch+1}, Training Loss: {loss.item()}')

    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_outputs = val_outputs.squeeze(-1)  # Ensure validation output matches target shape
        val_loss = criterion(val_outputs, y_val_tensor)
        print(f'Epoch {epoch+1}, Validation Loss: {val_loss.item()}')

    # Existing training and validation code...
    train_losses.append(loss.item())
    val_losses.append(val_loss.item())


plt.figure(figsize=(10, 5))
plt.plot(train_losses, label='Training Loss')
plt.plot(val_losses, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.show()

In [ ]:
model.eval()

with torch.no_grad():
    X_test_tensor = torch.tensor(X_test_seq, dtype=torch.float32).to(device)  # Ensure data is on the correct device
    predicted_output = model(X_test_tensor)
    predicted_output = predicted_output.cpu().numpy().flatten()  # Move predictions back to CPU and flatten if necessary

In [ ]:

dates = production_data['datetime'].values  # Ensure correct format
sequence_length = 24 

split_idx = int(len(dates) * 0.5)
# Prediction starts after the first 'sequence_length' entries in the test set
prediction_start_idx = split_idx + sequence_length

# Plotting
plt.figure(figsize=(15, 7))
plt.plot(dates[:split_idx], y_train, label='Training Data', color='blue')
plt.plot(dates[split_idx:], y_test, label='Actual Test Data', color='red')
plt.plot(dates[prediction_start_idx:], predicted_output, label='Predicted Data', color='green', linestyle='--')

plt.axvline(x=dates[split_idx], color='gray', linestyle='--', label='Start of Test Data')

# Formatting the date on the x-axis for better readability
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Rotation

plt.title('Training, Actual Test data, and Predicted Data on raw target values')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.show()

In [ ]:
original = hold['installed_capacity'][prediction_start_idx:] * predicted_output

plt.figure(figsize=(15, 7))
plt.plot(dates[split_idx:], hold['target'][split_idx:], label='Actual Test Data', color='red')
plt.plot(dates[prediction_start_idx:], original, label='Predicted Data', color='green', linestyle='--')
plt.axvline(x=dates[split_idx], color='gray', linestyle='--', label='Start of Test Data')
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()
plt.title('Predicted Data on original target values when multiplied back to original scale')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.show()

In [ ]:

scaled_predictions = hold['installed_capacity'][prediction_start_idx:] * predicted_output

plt.figure(figsize=(15, 7))
plt.plot(dates[split_idx:], hold['target'][split_idx:], label='Actual Test Data', color='red')
plt.plot(dates[prediction_start_idx:], scaled_predictions, label='Scaled Predicted Data', color='green', linestyle='--')

plt.axvline(x=dates[split_idx], color='gray', linestyle='--', label='Start of Test Data')

plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Rotate date labels for better readability

plt.title('Comparison of Actual Test Data and Scaled Predicted Data')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.show()

In [ ]:
full_scaled_predictions = np.empty_like(hold['target'])
full_scaled_predictions[:] = np.nan  # Initialize with NaNs
full_scaled_predictions[prediction_start_idx:] = hold['installed_capacity'][prediction_start_idx:] * predicted_output

plt.figure(figsize=(15, 7))
plt.plot(dates[:split_idx], hold['target'][:split_idx], label='Training Data', color='blue')
plt.plot(dates[split_idx:], hold['target'][split_idx:], label='Actual Test Data', color='red')

plt.plot(dates, full_scaled_predictions, label='Scaled Predicted Data', color='green', linestyle='--')
plt.axvline(x=dates[split_idx], color='gray', linestyle='--', label='Start of Test Data')

# Format the date on the x-axis for better readability
plt.gca().xaxis.set_major_locator(mdates.AutoDateLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%Y-%m-%d'))
plt.gcf().autofmt_xdate()  # Rotate date labels for better readability

plt.title('Full Timeline: Actual vs. Scaled Predicted Data')
plt.xlabel('Date')
plt.ylabel('Energy Production')
plt.legend()
plt.show()

In [ ]:
actual_test_values = hold['target'][prediction_start_idx:]

mae = mean_absolute_error(actual_test_values, scaled_predictions)
print(f"Mean Absolute Error for scaled predictions: {mae}")

mae = mean_absolute_error(y_test, predicted_output)
print(f"Mean Absolute Error for normalised predictions: {mae}")